This Jupyter file contains all the code necessary to run the model. 
1. Run code block 1. This trains the model. The dataset is available in Keras itself, and hence no installation is required.
2. Upload the image to be analyzed in code block 2.
3. Run blocks 3,4 to verify them. 
4. Code block 5 contains the output and must be run to check result of analysis (is it garment, if so what is it's colour)
5. For any further images, just run block 2 to add it, and directly run block 5.

Note: Please run this notebook on Google Colab or use the following link:
https://colab.research.google.com/drive/1avH3rFfal6y1n3sUcJ1xZqcFTBAcuGyr?usp=sharing

In [ ]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='sigmoid'))
	#sigmoid is used to analyze if it doesnt belong to any class

	
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# append scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(211)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(212)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

from random import randint
def predict_el():
    trainX, trainY, testX, testY = load_dataset()
    print(type(trainX)) ; return None
    trainX, testX = prep_pixels(trainX, testX)  
    n_folds = 5
    model = define_model()
    model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
    a,b = randint(0,len(testX)-1),randint(0,len(testX)-1)
    arr = model.predict(testX[a:a+10])
    for i in range(a,a+10):
        pyplot.imshow(testX[i].reshape(28,28), cmap=pyplot.get_cmap('gray'))
        pyplot.show()
        print(arr[i-a])
trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)  
model = define_model()
model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)

# entry point, run the test harness
#predict_elmnt(arr)

4431872/4422102 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
from google.colab import files
uploaded = files.upload()
filename = None
for item in uploaded.keys():
  filename = item
  break
def place_holder():
  return filename

Saving shrt.jpg to shrt.jpg


In [ ]:
from PIL import Image
from numpy import asarray
from skimage import io, color

def is_garment():
    filename = place_holder()
    image = Image.open(filename)
    #print(filename)
    resized = image.resize((28,28))
    resized.save("resize.png")
    lina_color = io.imread("resize.png")
    lina_gray = color.rgb2gray(lina_color)

    img_arr = lina_gray.reshape((1,28,28,1))
    tens = K.constant(img_arr)
    arr = model(tens)
    arr = arr.numpy()[0]
    max_prob = max(arr) ; min_prob = min(arr)
    isGar = None
    if round(max_prob,2)>=0.96 and round(min_prob,2)<=0.8:
        isGar = True
    else:
        isGar = False
    return isGar

In [ ]:
import cv2
from math import exp,log

color_lst = [(0,0,0),(255,255,255),(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(0,128,255),(255,0,127),(128,0,255)]
color_names = ["black","white","blue","green","red","cyan","pink","yellow","orange","violet","magenta"]
dct = {} ; count = 0 ; num_dct = {}
for item in color_lst:
  dct[item] = color_names[count]
  count+=1
for item in color_lst:
  num_dct[item] = 0

def predict_color():
  #def predict_color(name):
  def predColour(arr):
    def dist(x,y):
      val = (x[0]-y[0])**2+(x[1]-y[1])**2+(x[2]-y[2])**2
      return val
    lst = [dist(item,arr) for item in color_lst]
    return color_lst[lst.index(min(lst))]

  filename = place_holder()
  temp = cv2.imread(filename)
  img = cv2.resize(temp,(200,200),interpolation = cv2.INTER_AREA) #square
  n,b,_ = img.shape

  for i in range(n):
    for j in range(n):
      img[i][j] = predColour(img[i][j])

  #from google.colab.patches import cv2_imshow
  #cv2_imshow(img)

  def weight(x,y,n):
    std_dev = 2.5
    a = abs(x-n/2)/(std_dev*2) ; b = abs(y-n/2)/std_dev
    wei = exp(-1*a**2/2)*exp(-1*b**2/2)
    return wei
  num1,num2,num3,den = 0,0,0,0
  for i in range(n):
    for j in range(n):
      wei = weight(i+1,j+1,n)
      #dct[img[i][j]]+=weight
      total = num1+num2+num3+1
      num1+=wei*img[i][j][0]/total ; num2+=wei*img[i][j][1]/total ; num3+=wei*img[i][j][2]/total
      den+=wei/total
  num1/=den ; num2/=den ; num3/=den
  #print([num1,num2,num3])
  return dct[predColour([num1,num2,num3])]



In [ ]:
val = is_garment()
if val:
  col = predict_color()
  print("It is a garment of colour",col)
else:
  print("It is not a garment")

It is a garment of colour black
